# Homework 1. Выполнил Оганян Роберт

### Постановка задачи

In [154]:
#!c1.8
import numpy as np
import time
import numba

In [142]:
#!c1.8
def match_timestamps(timestamps_30fps, timestamps_60fps):
    """
    Let's say we have two cameras capturing the same scene. 
    One camera's frame rate is 60, antoher's is 30. However, due to a high CPU or 
    hard drive load the actual fps might differ from 30 and 60.
    
    This function matches the frames from the first camera to the second one, meaning
    that for each timestamp in timestamps_60fps it finds the index of closest one in timestamps_30fps.
    
    Inputs:
        - timestamps_30fps: sorted np.ndarray, dtype=np.floa64. Timestamps of the 30 fps camera.
        - timestamps_60fps: sorted np.ndarray, dtype=np.floa64. Timestamps of the 60 fps camera. 
    Outputs:
        - idxs: np.ndarray, dtype=np.int32. Indexes of timestamps matching.

    Example:
        - timestamps_30fps = np.array([0, 0.033, 0.066], dtype=np.float64)
        - timestamps_60fps = np.array([0, 0.016, 0.032, 0.048, 0.065], dtype=np.float64)
        - idxs = np.array([0, 0, 1, 1, 2], dtype=np.int32)
    
    This function must be as fast as possible on CPU from both algorithmic and Python-wise implementation perspectives.
    It must provide the correct result and must not fail on any realization of the described input. 
    You may use ANY lib or method you want up to writing a C++ wrapping callable from Python. 
    Consider you have multiple CPU cores.
    Send the best implementation of this function to asshilov@yandex.com or to tg @a_team2 before 23:59 24.11 
    in .py or .ipynb formats.
    Good luck!
    """
    idxs = np.ones(len(timestamps_60fps), dtype=np.int32) # replace this with your code
    return idxs

def get_sample_timestamps(fps, video_length_sec):
    timestamps = np.linspace(time.time(), time.time() + video_length_sec, video_length_sec * fps)
    timestamps += np.random.randn(len(timestamps)) / fps # emulate high CPU of drive load
    timestamps = np.sort(timestamps) # despite the load, timestamps have to be sorted
    return timestamps

video_length_sec = 1000 # optimize the implementation for the lengths in range 1000-4000 seconds
timestamps_30fps = get_sample_timestamps(30, video_length_sec)
timestamps_60fps = get_sample_timestamps(60, video_length_sec)
match_timestamps(timestamps_30fps, timestamps_60fps)

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

### Решение

Задание выполнено на датасфере на конфигурации c1.8. 8 физических ядер

#### Определим функцию для замера времени работы программы

Функция замеряет работу программы 10 раз, берет последние num_measures * (1-percent_skip/100) измерений и возвращает среднее время работы + результат вызова функции.

P.S: я вспомнил что можно просто использовать %%timeit. Ну ничего, опробуем и то, и то

In [143]:
#!c1.8
from timeit import default_timer as timer


def measure_time(callable, *args, max_time=5, num_measures=10, percent_skip=20, **kwargs):
    t0 = timer()
    result = callable(*args, **kwargs)
    time = timer() - t0

    if time > max_time:
        return (time, result)

    times = []
    for i in range(num_measures):
        t0 = timer()
        _ = callable(*args, **kwargs)
        times.append(timer() - t0)

    le = int(num_measures*percent_skip/100)
    ri = num_measures - le
    cnt = ri - le + 1
    sum_ = 0
    while le <= ri:
        sum_ += times[le]
        le += 1

    return (sum_ / cnt, result)

In [144]:
#!c1.8
video_length_sec = 1000 # optimize the implementation for the lengths in range 1000-4000 seconds
timestamps_30fps = get_sample_timestamps(30, video_length_sec)
timestamps_60fps = get_sample_timestamps(60, video_length_sec)

In [145]:
#!c1.8
timestamps_60fps.shape, timestamps_30fps.shape

((60000,), (30000,))

В дальнейшем будем называть длину первого массива (30 фпс) n1, n2 - длина второго (60 фпс)

#### 1. Наивное решение

Наивно: для каждого элемента второго массива проходим по всем элементам первого и ищем ближайший таймстемп. Ну и немного оптимизируя (не ассимптотически) заметим, что из-за упорядоченности данных нет смысла продолжать поиск, если найден хоть 1 таймстемп дальше, чем текущий лучший (криво написал, но думаю понятно). Сложность - **O(n1 * n2)**

In [146]:
#!c1.8
def match_timestamps_naive(timestamps_30fps, timestamps_60fps):   
    idxs = np.zeros(len(timestamps_60fps), dtype=np.int32)
    for idx, val_1 in enumerate(timestamps_60fps):
        cur_closest_val, cur_closest_idx = float('inf'), -1
        for jdx, val_2 in enumerate(timestamps_30fps):
            if abs(val_2 - val_1) < cur_closest_val:
                cur_closest_idx = jdx
                cur_closest_val = abs(val_2 - val_1) 
            elif cur_closest_idx != -1:
                break
        idxs[idx] = cur_closest_idx
    return idxs

Нам не очень важно как долго работает тупое решение, поэтому не буду запускать его 10 раз для честного теста. Запустим 1 раз и покажем, что оно работает чудовищно долго

In [147]:
#!c1.8
t0 = timer()
res_naive = match_timestamps_naive(timestamps_30fps, timestamps_60fps)
time_naive = timer() - t0

In [148]:
#!c1.8
time_naive

397.5587981860008

#### 2. Оптимальное решение
Заметим, что оба наши массива упорядочены, а если для i-ого элемента второго массива ответом является позиция j первого массива, то для i+1 элемента второго массива ответ не может лежать левее j. Функция ответа "унимодальна".

Решим эту задачу двумя указателями. Сложность - **O(n1+n2)**

In [149]:
#!c1.8
def match_timestamps_optimal(timestamps_30fps, timestamps_60fps):   
    idxs = np.zeros(len(timestamps_60fps), dtype=np.int32)
    right = 0
    for left, val_1 in enumerate(timestamps_60fps):
        cur_closest_val = float('inf')        
        while right < len(timestamps_30fps) and abs(timestamps_30fps[right] - val_1) < cur_closest_val:
            cur_closest_val=abs(timestamps_30fps[right] - val_1)
            right+=1
        right-=1
        idxs[left] = right
    return idxs

In [150]:
#!c1.8
time_optimal, res_optimal = measure_time(match_timestamps_optimal, timestamps_30fps, timestamps_60fps)

In [151]:
#!c1.8
%%timeit
match_timestamps_optimal(timestamps_30fps, timestamps_60fps)

114 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [152]:
#!c1.8
print('Время работы программы: ', time_optimal)
print('Совпадает решение с наивным: ', np.testing.assert_array_equal(res_naive, res_optimal, err_msg='Wrong result') is None)
print('Усорение работы путем лучшей ассимптотики (количество раз): ', time_naive / time_optimal)

Время работы программы:  0.1118685992862863
Совпадает решение с наивным:  True
Усорение работы путем лучшей ассимптотики (количество раз):  3553.801520019001


#### 3. Распараллелим оптимальное решение

Поделим блоки для потоков следующим образом

In [155]:
#!c1.8
blocks = []
cnt_of_elems_in_block = len(timestamps_60fps) //numba.get_num_threads()
for i in range(numba.get_num_threads()):
    blocks.append((cnt_of_elems_in_block * i, cnt_of_elems_in_block * (i+1)))
blocks   

[(0, 7500),
 (7500, 15000),
 (15000, 22500),
 (22500, 30000),
 (30000, 37500),
 (37500, 45000),
 (45000, 52500),
 (52500, 60000)]

Решение:

In [156]:
#!c1.8
@numba.njit(parallel=True)
def match_timestamps_optimal_paral(timestamps_30fps, timestamps_60fps):   
    idxs = np.zeros(len(timestamps_60fps), dtype=np.int32)
    blocks = []
    cnt_of_elems_in_block = len(timestamps_60fps) //numba.get_num_threads()
    for i in range(numba.get_num_threads()):
        blocks.append((cnt_of_elems_in_block * i, cnt_of_elems_in_block * (i+1)))
        
    for block_idx in numba.prange(numba.get_num_threads()):
        block = blocks[block_idx]
        right = 0
        for left in range(*block):
            val_1 = timestamps_60fps[left]
            cur_closest_val = 1e10        
            while right < len(timestamps_30fps) and abs(timestamps_30fps[right] - val_1) < cur_closest_val:
                cur_closest_val=abs(timestamps_30fps[right] - val_1)
                right+=1
            right-=1
            idxs[left] = right
    return idxs

In [157]:
#!c1.8
%%timeit
res_optimal_paral = match_timestamps_optimal_paral(timestamps_30fps, timestamps_60fps)

195 µs ± 78.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [158]:
#!c1.8
time_optimal_paral, res_optimal_paral = measure_time(match_timestamps_optimal_paral, timestamps_30fps, timestamps_60fps)

In [162]:
#!c1.8
print('Время работы программы: ', time_optimal_paral)
print('Совпадает решение с наивным: ', np.testing.assert_array_equal(res_naive, res_optimal_paral, err_msg='Wrong result') is None)
print('Усорение работы путем лучшей ассимптотики (количество раз): ', time_naive / time_optimal_paral)
print('Усорение работы путем распараллеливания: (количество раз)', time_optimal / time_optimal_paral)

Время работы программы:  0.0001552934305176937
Совпадает решение с наивным:  True
Усорение работы путем лучшей ассимптотики (количество раз):  2560049.0430321456
Усорение работы путем распараллеливания: (количество раз) 720.369167667658


Думаю, такое дикое усорение обусловлено тем, что в первый запуск намба компилирует код, а затем пользуется этим.

##### Идея для бОльшего ускорения:

При распараллеливании мы для каждого блока берем второй указатель как 0, потому что в отличие от последовательной версии, в параллельной версии нельзя наладить взаимодействие между вторыми указателями. Вместо того, чтобы делать указатели нулями, можно для первого элемента каждого блока найти ближайший таймстемп [**тернарным поиском**](https://e-maxx.ru/algo/ternary_search) (так как функция унимодальна на всем отрезке), и затем уже делать как с двуми указателями. Хоть этот шаг работает за O(logn1), ассимптотически все равно выйдет в итоге O(n1+n2). Однако такая оптимизация возможно даст небольшой выигрыш, но проверять я это не буду :))

#### Итог

1. Мы получили алгоритмически быстрое решение O(n1+n2)
2. Мы получили питонически быстрое решение, распараллелив код с помощью numba
3. Мы получили правильное решение, сравнив результаты работы с наивным решением

Итоговая функция:

In [177]:
#!c1.8
@numba.njit(parallel=True)
def match_timestamps(timestamps_30fps, timestamps_60fps):   
    idxs = np.zeros(len(timestamps_60fps), dtype=np.int32)
    blocks = []
    cnt_of_elems_in_block = len(timestamps_60fps) //numba.get_num_threads()
    for i in range(numba.get_num_threads()):
        blocks.append((cnt_of_elems_in_block * i, cnt_of_elems_in_block * (i+1)))
        
    for block_idx in numba.prange(numba.get_num_threads()):
        block = blocks[block_idx]
        right = 0
        for left in range(*block):
            val_1 = timestamps_60fps[left]
            cur_closest_val = 1e10        
            while right < len(timestamps_30fps) and abs(timestamps_30fps[right] - val_1) < cur_closest_val:
                cur_closest_val=abs(timestamps_30fps[right] - val_1)
                right+=1
            right-=1
            idxs[left] = right
    return idxs